### Data Preprocessing

In [ ]:
CUDA_VISIBLE_DEVICES=1 python inference_on_a_image.py \
-c config_model/UniPose_SwinT.py \
-p weights/unipose_swint.pth \
-i ~/imgs/o.png \
-o "/l/users/fathinah.izzati/work/X-Pose/output" \
-t "cartoon"

\ (e.g., "person", "face", "left hand", "horse", "car", "skirt", "table")
-k "keypoint_skeleton_text" (If necessary, please select an option from the 'predefined_keypoints.py' file.)


pip install torch==2.1.2+cu118 torchvision==0.16.2+cu118      -f https://download.pytorch.org/whl/torch_stable.html


### Splitting data into 10 sec clips (activate Synchformer)

In [6]:
import ffmpeg
import os
def chuck_video(video_path, output_folder):
    '''
    Chunk a full video into 10 seconds clips with 5 fps and 16000 Hz audio.
    '''
    segment_duration = 10.0  # segment duration in seconds
    # Get video duration in seconds using ffmpeg
    probe = ffmpeg.probe(video_path)
    video_duration = float(probe['format']['duration'])
    output_paths = []
    if video_duration > segment_duration:
        if os.path.exists(output_folder):
            output_paths = [os.path.join(output_folder, name) for name in os.listdir(output_folder)]
            print(f"The folder '{output_folder}' was created.")
        else:
            for i, start_time in enumerate(range(0, int(video_duration), int(segment_duration))):
                if int(video_duration)-start_time>=segment_duration:
                    os.makedirs(output_folder+f"/segment_{i+1:03d}")
                    segment_path = os.path.join(output_folder+f"/segment_{i+1:03d}",'video.mp4')
                    ffmpeg.input(video_path, ss=start_time, t=segment_duration).output(
                        segment_path, vf="fps=5", ar=32000
                    ).run(overwrite_output=True)
                    output_paths.append(segment_path)
            print("Video segmentation complete! Without the non-10 s element")  
    else:
        print("No need to segment video!")
        output_paths.append(video_path)

    return output_paths

In [7]:
file_name = '/l/users/fathinah.izzati/coco-mulla-repo/demo/input/tnj/tnj_test.mp4'
output_folder = '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test'
# file_name = '/l/users/fathinah.izzati/Synchformer/output/tnj_test.mp4/segment_001.mp4'
chuck_video(file_name, output_folder)

ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/fathinah.izzati/miniconda3/envs/synchformer --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/l/users/fathinah.izzati/coco-mulla-repo/demo/input/tnj/tnj_test.mp4':
  Metadata:
    maj

Video segmentation complete! Without the non-10 s element


frame=   50 fps=0.0 q=1.6 Lsize=     398kB time=00:00:10.01 bitrate= 325.6kbits/s speed=29.1x    
video:235kB audio:159kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.918490%
[aac @ 0x5556dd061c00] Qavg: 432.952


['/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_001/video.mp4',
 '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_002/video.mp4',
 '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_003/video.mp4',
 '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_004/video.mp4',
 '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_005/video.mp4',
 '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_006/video.mp4',
 '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_007/video.mp4',
 '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_008/video.mp4',
 '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_009/video.mp4',
 '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_010/video.mp4',
 '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_011/video.mp4',
 '/l/users/fathinah.izzati/coco-

### Create video embedding

In [34]:
vid_name = '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_029/video.mp4'


In [44]:
from torchvision.io.video import read_video
from torchvision.models.video import r3d_18, R3D_18_Weights
import torch
import os
import numpy as np

for k in os.listdir('/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/'):
    print(k)
    folder_name = '/l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/'+k+'/'
    file_name = folder_name+'video.mp4'
    vid, _, info = read_video(vid_name, output_format="TCHW")
    fps = info['video_fps']
    total_frames = vid.shape[0]

    weights = R3D_18_Weights.DEFAULT
    model = r3d_18(weights=weights)
    model.eval()
    model.fc = torch.nn.Identity()
    preprocess = weights.transforms()

    embeddings = []


    for i in range(total_frames):
        vid_cropped = vid[i:int(i+fps)]
        batch = preprocess(vid_cropped).unsqueeze(0)
        with torch.no_grad():
            embedding = model(batch)
        embeddings.append(embedding)
    combined_embeddings = torch.cat(embeddings, dim=0)
    output_path = os.path.join(folder_name,'video_emb.npy')
    np.save(output_path, combined_embeddings.numpy())
    print('combined_embedding', combined_embeddings.shape,'is saved to', output_path)

segment_025


/home/fathinah.izzati/miniconda3/envs/cocomulla/lib/python3.11/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


combined_embedding torch.Size([50, 512]) is saved to /l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_025/video_emb.npy
segment_017
combined_embedding torch.Size([50, 512]) is saved to /l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_017/video_emb.npy
segment_004
combined_embedding torch.Size([50, 512]) is saved to /l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_004/video_emb.npy
segment_003
combined_embedding torch.Size([50, 512]) is saved to /l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_003/video_emb.npy
segment_009
combined_embedding torch.Size([50, 512]) is saved to /l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_009/video_emb.npy
segment_010
combined_embedding torch.Size([50, 512]) is saved to /l/users/fathinah.izzati/coco-mulla-repo/demo/output/tnj_test/segment_010/video_emb.npy
segment_028
combined_embedding torch.Size([50, 512]) is saved to /l/users/fathinah.izzati/coco